# Imports
The code imports several libraries to work correctly. The libraries are as follows:

- tqdm is a library that provides a progress bar for loops and iteration, which is helpful for keeping track of the progress of long-running processes.
- PIL (Python Imaging Library) is a library used for opening, manipulating, and saving image files.
- The pickle library allows the user to serialize and deserialize Python objects, meaning that the user can save a Python object to a file and then load it back later.
- The os library provides a way of using operating system dependent functionality like reading or writing to the file system.
- The zipfile library provides functionality to create, read, write, append, and list a ZIP file.
- requests library is used for sending HTTP requests to web servers and downloading content.
- functools is a module that implements higher-order functions. Higher-order functions are functions that take other functions as inputs, or that return functions as output.
- pathlib is a library for working with file paths, which is part of the Python Standard Library starting from Python 3.4.
- Finally, tqdm.auto automatically enables or disables the progress bar depending on the context of use, so that it only shows when the output is connected to a terminal or not.

In [1]:
!pip install nest_asyncio pickle zipfile requests functools pathlib tqdm json sqlite3 pandas PIL nest_asyncio

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:

import pickle
import os
import zipfile
import requests
import functools
import pathlib
from tqdm import tqdm
import json
import sqlite3
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS
import nest_asyncio

# Settings base variables and paths
For this project, we used the unsplash dataset, which is a large-scale image dataset. The dataset contains over 25,000 images.
The code sets the base variables and paths for the project. The variables are as follows:

In [11]:
# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
config_path = os.path.join(output_path, "config")

list_of_paths = [output_path, images_path, metadata_path, config_path]

# Set the base URL for the dataset
dataset_url = "https://unsplash-datasets.s3.amazonaws.com/lite/latest/unsplash-research-dataset-lite-latest.zip"
# metadata mode (used to save metadata)
metadata_mode = "sqlite"

# Create folder structure
The code creates the folder structure for the project. The folder structure is as follows:
- output
    - images
    - metadata
    - config

This method creates a folder with the given path if it doesn't already exist, It also outputs a message to inform the user if the folder was created or if it already exists.
This is useful for organizing and managing files in a project. By creating a folder to store data and resources, it keeps the working directory tidy and makes it easier to locate files. Additionally, by checking if the folder exists before creating it, it prevents the program from overwriting existing data or throwing an error.

In [12]:
def create_folder(path):
    """
    This function creates a folder at the specified path.
    If the folder already exists, it will print a message saying so.
    If there is an error creating the folder, it will print the error message.

    Parameters:
        :param path (str): The path of the folder to be created.

    Returns:
    None
    """
    try:
        # Use os.mkdir to create the folder at the specified path
        os.mkdir(path)
        print(f"Folder {path} created")
    except FileExistsError:
        # If the folder already exists, print a message saying so
        print(f"Folder {path} already exists")
    except Exception as e:
        # If there is an error creating the folder, print the error message
        print(f"Error creating folder {path}: {e}")

# Create the folder structure
This method initializes a list of folders by calling the create_folder method for each folder in the list.
The purpose of this method is to make sure that all necessary folders exist before the program continues its execution.
If a folder does not exist, the create_folder method will create it. If a folder already exists, the method will simply print a message indicating that the folder already exists. In case of any other error, the method will print the error message.

In [13]:
def init_folder(folder_names: list):
    for folder_name in folder_names:
        create_folder(folder_name)

In [14]:
init_folder(list_of_paths)

Folder ../output already exists
Folder ../output/images already exists
Folder ../output/metadata already exists
Folder ../output/config already exists


# Define methods for downloading the dataset
The following code block is a method to download a file from a given URL and save it to a specified filename.

The method starts by creating a session (s = requests.Session()) and then mounting it to the URL (s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))). This sets the maximum number of retries to 3 if the connection to the URL fails.

Then, the method makes a GET request to the URL (r = s.get(url, stream=True, allow_redirects=True)) and checks if it returns a successful response (r.raise_for_status()). If there was an HTTP error during the request, the error message is printed (print(f"HTTP error occurred while downloading dataset: {e}")).

The method also checks the file size specified in the response headers and assigns it to the variable file_size (file_size = int(r.headers.get('Content-Length', 0))). If the file size is 0, a default value of "(Unknown total file size)" is assigned to the variable desc; otherwise, the variable desc is left empty.

Next, the method resolves the file path and creates a directory if it doesn't already exist (path.parent.mkdir(parents=True, exist_ok=True)). The method then creates a tqdm progress bar to show the download progress (with tqdm.tqdm(total=file_size, unit='B', unit_scale=True, desc=desc) as pbar:).

Finally, the method writes the contents of the file to disk in chunks (for chunk in r.iter_content(chunk_size=1024):), updating the progress bar for each chunk that is written to disk (pbar.update(len(chunk))). If an error occurred during the download, a message with the error is printed (print(f"Error occurred while downloading dataset: {e}")). The file path is returned when the method is finished.

In [15]:
def download(url, filename):
    """
    This download a file from a given URL and save it to a specified filename.

    Parameters:
        :param url (str): The URL of the file to be downloaded.
        :param filename (str): The filename to save the file as.

    Returns:
    path (str): The path of the downloaded file.
    """
    try:
        # Create a session object to persist the state of connection
        s = requests.Session()
        # Retry connecting to the URL up to 3 times
        s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))
        # Send a GET request to the URL to start the download
        r = s.get(url, stream=True, allow_redirects=True)
        # Raise an error if the response is not 200 OK
        r.raise_for_status()
        # Get the file size from the Content-Length header, default to 0 if not present
        file_size = int(r.headers.get('Content-Length', 0))
        # Get the absolute path to the target file
        path = pathlib.Path(filename).expanduser().resolve()
        # Create parent directories if they don't exist
        path.parent.mkdir(parents=True, exist_ok=True)
        # Set the description to display while downloading, "(Unknown total file size)" if file size is 0
        desc = "(Unknown total file size)" if file_size == 0 else ""
        # Enable decoding the response content
        r.raw.read = functools.partial(r.raw.read, decode_content=True)
        # Use tqdm to display the download progress
        with tqdm(total=file_size, unit='B', unit_scale=True, desc=desc) as pbar:
            # Open the target file in binary write mode
            with path.open("wb") as f:
                # Write each chunk of data from the response to the file
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
                    pbar.update(len(chunk))
        # Return the path to the downloaded file
        return path
    # Handle HTTP error if the response is not 200 OK
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred while downloading dataset: {e}")
    # Handle any other exceptions that might occur while downloading the file
    except Exception as e:
        print(f"Error occurred while downloading dataset: {e}")

# Download the dataset
The following code block downloads the dataset from the URL and saves it to the specified filename. The method also prints a message to inform the user that the download is complete.

In [16]:
def download_dataset(dataset_url, image_path):
    """
    Downloads the dataset from the given URL, unzips it, and stores the images in the specified image path.

    Args:
        :param dataset_url (str): URL of the dataset to be downloaded
        :param image_path (str): Path to store the images after unzipping the dataset
    """
    # Check if the dataset has already been downloaded
    # Check if the archive.zip file exists or if the images folder is empty
    if not os.path.exists('archive.zip'):
        # Download the dataset from the given url
        download(dataset_url, 'archive.zip')
        print("Dataset downloaded!")
        try:
            # Extract the contents of the archive.zip to the specified image path
            with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
                zip_ref.extractall(image_path)
            print("Dataset unzipped")
        except Exception as e:
            print(f"Error occurred while unzipping dataset: {e}")
        try:
            # Remove the archive.zip file
            os.remove('archive.zip')
            print("archive.zip removed")
        except Exception as e:
            print(f"Error occurred while removing archive.zip: {e}")

In [9]:
download_dataset(dataset_url, images_path)


  9%|▉         | 58.1M/632M [01:04<10:37, 901kB/s]  


KeyboardInterrupt: 

In [17]:
nest_asyncio.apply()

In [18]:
# Read photo.tsv file in images folder
photo_df = pd.read_csv(os.path.join(images_path, 'photos.tsv000'), sep='\t')
# read photo_image_url column and photo_id in index
photo_df = photo_df[['photo_id', 'photo_image_url']]
print(photo_df.head())

      photo_id                                    photo_image_url
0  XMyPniM9LF0  https://images.unsplash.com/uploads/1411949294...
1  rDLBArZUl1c  https://images.unsplash.com/photo-141633941111...
2  cNDGZ2sQ3Bo  https://images.unsplash.com/photo-142014251503...
3  iuZ_D1eoq9k  https://images.unsplash.com/photo-141487280988...
4  BeD3vjQ8SI0  https://images.unsplash.com/photo-141700759404...


In [19]:
import asyncio
import aiohttp
import time


async def download_image(session, url, i, err_cnt=None):
    if err_cnt is None:
        err_cnt = 0
    try:
        async with session.get(url) as response:
            filename = os.path.join(images_path, "image_" + str(i) + ".jpg")
            with open(filename, 'wb') as f:
                f.write(await response.content.read())
            print(f"Downloaded {url} to {filename} idx: {i}")
    except aiohttp.ClientError as e:
        print(f"Error occurred while downloading {url}: {e}")
        if err_cnt == 10:
            return
        await asyncio.sleep(10)
        err_cnt += 1
        await download_image(session, url, i, err_cnt)


async def download_images(image_urls, images_ids):
    async with aiohttp.ClientSession() as session:
        tasks = []
        semaphore = asyncio.Semaphore(2000)
        # add index
        for i, url in enumerate(image_urls):
            await semaphore.acquire()
            task = asyncio.ensure_future(download_image(session, url, images_ids[i]))
            task.add_done_callback(lambda x: semaphore.release())
            tasks.append(task)
    await asyncio.gather(*tasks)


image_urls = photo_df['photo_image_url'].values.tolist()
images_ids = photo_df['photo_id'].values.tolist()
# filter by looking if the image already exist in fact of the image_id is already in the folder
# Loop on the image_id and check if the image exist in the folder
image_urls = [url for url, image_id in zip(image_urls, images_ids) if
              not os.path.exists(os.path.join(images_path, "image_" + str(image_id) + ".jpg"))]
print(f"Number of images to download: {len(image_urls)}")

Number of images to download: 25000


In [20]:

loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
# Split the list of image urls into chunks of 5000 and add a timeout of 30 seconds
chunks = [image_urls[i:i + 5000] for i in range(0, len(image_urls), 5000)]
start_t = time.time()
for i, chunk in enumerate(chunks):
    start = time.time()
    try:
        loop.run_until_complete(download_images(image_urls, images_ids))
        # Wait for all running tasks to complete
        pending = asyncio.Task.all_tasks(loop=loop)
        gathered = asyncio.gather(*pending, return_exceptions=True)
        loop.run_until_complete(gathered)
    finally:
        loop.close()
        asyncio.sleep(30)
        print(f"[Chunk {i}] Downloaded {len(chunk)} images in {time.time() - start} seconds")

print(f'Downloaded {len(image_urls)} images in {time.time() - start_t} seconds')

Downloaded https://images.unsplash.com/photo-1501275578-61eff411d8bf to ../output/images/image_6J_qm8DJWnY.jpg idx: 6J_qm8DJWnY
Downloaded https://images.unsplash.com/photo-1428550670225-15f007f6f1ba to ../output/images/image_cGe1PV_yEso.jpg idx: cGe1PV_yEso
Downloaded https://images.unsplash.com/reserve/m6rT4MYFQ7CT8j9m2AEC_JakeGivens%20-%20Sunset%20in%20the%20Park.JPG to ../output/images/image_ocwmWiNAWGs.jpg idx: ocwmWiNAWGs
Downloaded https://images.unsplash.com/photo-1505962372142-dcf8ddcf4c72 to ../output/images/image_WMLH0gatK50.jpg idx: WMLH0gatK50
Downloaded https://images.unsplash.com/photo-1502977249166-824b3a8a4d6d to ../output/images/image_kkJuQhp9Kw0.jpg idx: kkJuQhp9Kw0
Downloaded https://images.unsplash.com/photo-1492644233019-549102aa44f7 to ../output/images/image_sQgW0mQ1CY0.jpg idx: sQgW0mQ1CY0
Downloaded https://images.unsplash.com/photo-1420142515034-86cc8c508475 to ../output/images/image_cNDGZ2sQ3Bo.jpg idx: cNDGZ2sQ3Bo
Downloaded https://images.unsplash.com/photo

/var/folders/3m/wyqs41z16v53xn4gssj1tz2h0000gn/T/ipykernel_7232/3009411489.py:16: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(30)


[Chunk 0] Downloaded 5000 images in 3777.8030626773834 seconds


TimeoutError: 

In [22]:

# Remove all files except images
for file in os.listdir(images_path):
    if file.endswith('.jpg'):
        continue
    else:
        try:
            # Don't delete TERMS.md
            if file == 'TERMS.md':
                continue
            os.remove(os.path.join(images_path, file))
        except Exception as e:
            continue

# Define methods to get all the image paths
The get_all_images method is used to retrieve all images present in the specified image path. It uses the os.walk function to traverse through all subdirectories within the image path and collects the file names that end with either '.png' or '.jpg' extensions. The full path of each image is then generated by joining the root directory and the file name. The method returns a list of all images' full paths. In case of any error, an error message is printed and an empty list is returned.

In [23]:
def get_all_images(path):
    """Get all images from the given path.

    Args:
    param: image_path (str): path to the directory containing the images.

    Returns:
    - list: a list of full path to all the images with png or jpg extensions.
    - empty list: an empty list if an error occurred while fetching images.
    """
    try:
        # use os.walk to traverse all the subdirectories and get all images
        return [os.path.join(root, name)
                for root, dirs, files in os.walk(path)
                for name in files
                if name.endswith((".png", ".jpg"))]
    except Exception as e:
        # return an empty list and log the error message if an error occurred
        print(f"An error occurred while fetching images: {e}")
        return []

# Define methods to set a checkpoint system

## Method: create_checkpoint
This method is used to create a checkpoint file containing the latest processed image.

The method first tries to open a file named checkpoint.txt in write mode. Then it writes the latest_file into it. If any error occurs during this process, the error message is printed with the message "An error occurred while creating checkpoint: [error message]".

In [24]:
# Method to create a checkpoint with the latest file
def create_checkpoint(latest_file):
    """
   Creates a checkpoint file containing the latest processed file name.

   Parameters:
       :param latest_file (str): The name of the latest processed file.

   Returns:
       None
   """
    try:
        # Open a file in write mode
        with open('checkpoint.txt', 'w') as f:
            # Write the latest file to the checkpoint
            f.write(latest_file)
    except Exception as e:
        # Print error message
        print(f"An error occurred while creating checkpoint: {e}")

## Method: load_checkpoint
This method is used to load the checkpoint file to get the latest processed image.

The method first checks if the checkpoint file exists by verifying the existence of checkpoint.txt. If the checkpoint file exists, it opens the file in read mode, reads the content, and returns it. If the checkpoint file does not exist, it prints "Checkpoint not found" and returns None. If any error occurs during this process, the error message is printed with the message "An error occurred while loading checkpoint: [error message]".

In [25]:
# Method to load a checkpoint
def load_checkpoint():
    """
    Loads the checkpoint file if it exists.

    Returns:
        str: The name of the latest processed file, None if checkpoint file not found.
    """
    try:
        # Check if checkpoint exists
        if os.path.exists('checkpoint.txt'):
            # Open the checkpoint in read mode
            with open('checkpoint.txt', 'r') as f:
                # Return the contents of the checkpoint
                return f.read()
        else:
            # Print message if checkpoint not found
            print("Checkpoint not found")
            return None
    except Exception as e:
        # Print error message
        print(f"An error occurred while loading checkpoint: {e}")
        return None

## Method: remove_checkpoint
This method is used to remove the checkpoint file.

The method first checks if the checkpoint file exists by verifying the existence of checkpoint.txt. If the checkpoint file exists, it removes the file and prints "Checkpoint removed successfully". If the checkpoint file does not exist, it prints "Checkpoint not found". If any error occurs during this process, the error message is printed with the message "An error occurred while removing checkpoint: [error message]".

In [26]:
# Method to remove a checkpoint
def remove_checkpoint():
    """
    Removes the checkpoint file if it exists.

    Returns:
        None
    """
    try:
        # Check if checkpoint exists
        if os.path.exists('checkpoint.txt'):
            # Remove the checkpoint
            os.remove('checkpoint.txt')
            # Print success message
            print("Checkpoint removed successfully")
        else:
            # Print message if checkpoint not found
            print("Checkpoint not found")
    except Exception as e:
        # Print error message
        print(f"An error occurred while removing checkpoint: {e}")

# Define methods to set test dataset
The set_test_dataset method is used to limit the number of images in a given directory. The method takes two arguments, image_path and amount. The image_path argument specifies the directory containing the images, while the amount argument specifies the number of images that should be kept in the directory. The method works by looping through all the files in the directory and removing any file if the number of files exceeds the value specified by amount. If an error occurs during the execution of the method, an error message will be printed.

In [27]:
def set_test_dataset(image_path, amount=100):
    """
    This function removes all images from the given image_path except the first amount images.

    Parameters:
    :param image_path (str): The path to the image folder.
    :param amount (int, optional): The number of images to keep in the folder. Defaults to 100.

    Returns:
    None
    """
    try:

        # loop through the images in the directory using tqdm
        for root, dirs, files in os.walk(image_path, topdown=False):
            for name in tqdm(files, desc="Removing images"):
                # check if the number of images is greater than the amount specified
                if len(os.listdir(image_path)) > amount:
                    # remove the image if the number of images is greater than the specified amount
                    os.remove(os.path.join(root, name))
        # print message indicating that images were removed successfully
        print("All images removed except " + str(amount) + " images")
    except Exception as e:
        # print error message if there was an error during removal of images
        print(f"An error occurred while setting test dataset: {e}")

# Define methods to process dataset
The arrange_dataset method performs several tasks to clean up and organize a directory containing images. The steps are as follows:

- Get a list of all the image files in the directory using the get_all_images method.

- Load the latest checkpoint using the load_checkpoint method, which is used to keep track of the last image file processed.

- For each image file in the list, the method checks if it is the same as the latest checkpoint. If it is, it skips it and sets the checkpoint to None. If the checkpoint is not None, the method skips it. If the checkpoint is None, it moves the file to the image_path directory using the os.rename method. It also creates a checkpoint using the create_checkpoint method to keep track of the last image processed.

- After processing all the image files, the method removes the checkpoint using the remove_checkpoint method.

- The method then removes all subdirectories in the image_path directory using os.rmdir.

- If the is_test argument is set to True, the method calls the set_test_dataset method to remove all images except for a certain number specified by the amount argument (defaults to 100).

The method includes a try-except block to catch any errors that may occur while processing the images. If an error occurs, it prints a message indicating that an error occurred while arranging the dataset.


In [28]:
def arrange_dataset(image_path, is_test=False):
    """
    Arrange the dataset stored in `image_path`.

    :param image_path: path to the dataset folder.
    :param is_test: If True, the dataset will be set to a test set with only 100 images.
    """
    try:
        # Get a list of all images in the path
        img_files = get_all_images(image_path)
        # Load the last checkpoint if it exists
        checkpoint = load_checkpoint()
        # Iterate over all image files
        for file in tqdm(img_files, desc="Moving all file to images folder"):
            # Check if the current file matches the checkpoint
            if checkpoint == file:
                # If it does, reset the checkpoint
                checkpoint = None
                continue
            # If the checkpoint is not None, skip this file
            elif checkpoint is not None:
                continue
            # If neither of the above conditions are met, move the file
            else:
                os.rename(file, os.path.join(image_path, os.path.basename(file)))
                # Create a new checkpoint after moving the file
                create_checkpoint(file)
        # Print a message indicating that all files have been moved
        print("All files moved to images folder")
        # Remove the checkpoint since all files have been moved
        remove_checkpoint()

        # Remove all subfolders in the image path
        for root, dirs, files in os.walk(image_path, topdown=False):
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        # Print a message indicating that all subfolders have been removed
        print("All subfolders removed")

        print(is_test)
        # If is_test is True, set the test dataset
        if is_test:
            print("Setting test dataset 13&3" + image_path)
            set_test_dataset(image_path)
            print("Test dataset set successfully")
    # Catch any exceptions that may occur
    except Exception as e:
        print("An error occurred while arranging the dataset: ", e)

In [29]:
arrange_dataset(images_path, False)

Checkpoint not found


Moving all file to images folder: 100%|██████████| 4490/4490 [00:00<00:00, 12986.87it/s]

All files moved to images folder
Checkpoint removed successfully
All subfolders removed
False


# Define methods to get metadata
The method get_metadata is used to extract metadata information from an image file and return it in a dictionary format. The method takes two parameters: img_file and image_path. img_file is the file name of the image and image_path is the path to the directory where the image is stored.

The method uses the Image module from the Python Imaging Library (PIL) to open the image file. Then it gets the EXIF data from the image and stores it in a dictionary along with other metadata information such as the file name, size, height, width, format, and mode of the image.

If an error occurs while processing the image file, the method will print an error message and return None. Otherwise, the method returns the metadata information in the form of a dictionary.


In [30]:
def get_metadata(img_file):
    """
    This function extracts metadata information from an image file and returns it in a dictionary format.

    Parameters:
    img_file (str): The file name of the image.
    image_path (str): The path to the directory where the image is stored.

    Returns:
    dict: A dictionary containing the metadata information of the image. If an error occurs, the function returns None.
    """
    try:
        with Image.open(img_file) as image:
            metadata = image._getexif()

        # replace the key with the name of the tag
        metadata = {TAGS.get(key, key): value for key, value in metadata.items()}
        # get image width and height in pixels
        width, height = image.size
        # add the image width and height to the metadata dictionary
        metadata['width'] = width
        metadata['height'] = height

    except Exception as e:
        # print an error message if an error occurs
        print(f"An error occurred while processing {img_file}: {str(e)}")
        return None

    # return the metadata information
    return metadata



# Define the method to save the metadata
The method save_metadata is used to save the metadata information of an image in either pickle or json format. The method takes in 4 arguments:

- metadata (dict): The metadata information of an image.
- img_name (str): The file name of the image.
- metadata_path (str): The path to the directory where the metadata will be saved.
- save_format (str): The format in which the metadata will be saved. The default is 'pickle'.

The method first checks if the save_format argument is equal to 'pickle'. If it is, it uses the pickle module to save the metadata information in pickle format. The file name of the saved metadata is created by joining the metadata_path with the base name of the image file (obtained using os.path.basename) and the '.pickle' extension.

If the save_format argument is equal to 'json', the method uses the json module to save the metadata information in json format. The file name of the saved metadata is created by joining the metadata_path with the base name of the image file (obtained using os.path.basename) and the '.json' extension.

If the save_format argument is neither 'pickle' nor 'json', the method raises a ValueError with the message "Invalid save format".

The method uses a try-except block to catch any exceptions that may occur during the saving of the metadata. If an error occurs, it prints an error message that includes the image file name and the error message.


In [31]:
def save_metadata(metadata, img_name, metadata_path, save_format='pickle'):
    """
    This function saves the metadata information of an image in either pickle or json format.
    Parameters:
    metadata (dict): The metadata information of an image.
    img_name (str): The file name of the image.
    metadata_path (str): The path to the directory where the metadata will be saved.
    save_format (str): The format in which the metadata will be saved. The default is 'pickle'.

    Returns:
    None
    """
    try:
        if save_format == 'pickle':
            # save the metadata in pickle format
            with open(os.path.join(metadata_path, os.path.splitext(os.path.basename(img_name))[0] + '.pickle'),
                      'wb') as f:
                pickle.dump(metadata, f)
        elif save_format == 'json':
            # save the metadata in json format
            with open(os.path.join(metadata_path, os.path.splitext(os.path.basename(img_name))[0] + '.json'), 'w') as f:
                json.dump(metadata, f)
        elif save_format == 'sqlite':
            # Get only the file name of the image
            img_name = os.path.basename(img_name)
            # Open a connection to the database
            conn = sqlite3.connect(os.path.join(metadata_path, 'metadata.db'))
            # Create a cursor
            c = conn.cursor()
            # Create a table if it doesn't exist : filename, key, value
            c.execute('''CREATE TABLE IF NOT EXISTS metadata (filename text, key text, value text)''')
            # Insert the metadata into the table
            for key, value in metadata.items():
                # Convert key, value to string
                key = str(key)
                value = str(value)
                # Check if the key is already in the table
                c.execute("SELECT * FROM metadata WHERE filename=? AND key=?", (img_name, key))
                # If the key is already in the table, update the value
                if c.fetchone():
                    c.execute("UPDATE metadata SET value=? WHERE filename=? AND key=?", (value, img_name, key))
                    # Commit the changes
                    conn.commit()
                # If the key is not in the table, insert the key, value pair
                else:
                    c.execute("INSERT INTO metadata VALUES (?, ?, ?)", (img_name, key, value))
                    # Commit the changes
                    conn.commit()
            # Close the connection
            conn.close()
        else:
            raise ValueError("Invalid save format")
    except Exception as e:
        # print an error message if an error occurs
        print(f"An error occurred while saving metadata for {img_name}: {str(e)}")


# Get all the metadata
The method get_all_metadata is used to extract metadata information from all images in a directory and save the metadata information in either pickle or json format. The method takes in 3 arguments:
- image_path (str): The path to the directory where the images are stored.
- metadata_path (str): The path to the directory where the metadata will be saved.
- save_format (str): The format in which the metadata will be saved. The default is 'pickle'.

In [32]:
def get_all_metadata(image_path, metadata_path, save_format='pickle'):
    """
    This function extracts metadata from all images in a directory and saves the metadata information in either pickle or json format.
    Parameters:
    image_path (str): The path to the directory where the images are stored.
    metadata_path (str): The path to the directory where the metadata will be saved.

    Returns:
    None
    """
    # Get a list of all images in the directory
    img_files = get_all_images(image_path)
    # Create a progress bar to track the progress of processing all images
    checkpoint = load_checkpoint()
    for img in tqdm(img_files, desc="Get all metadata of the images and save it"):
        # Check if the metadata extraction process was interrupted previously
        if checkpoint == img:
            # If the checkpoint is found, set it to None and continue processing the remaining images
            checkpoint = None
            continue
        elif checkpoint is not None:
            # If the checkpoint is not None and not equal to the current image, continue to the next image
            continue
        else:
            # Extract the metadata of the current image
            metadata = get_metadata(img)
            if metadata:
                # Save the metadata of the current image
                save_metadata(metadata, img, metadata_path, save_format)
                # Create a checkpoint to track the progress of the metadata extraction process
                create_checkpoint(img)

    # Remove the checkpoint file once all metadata have been extracted and saved
    remove_checkpoint()


In [33]:
get_all_metadata(images_path, metadata_path, save_format=metadata_mode)

Checkpoint not found


Get all metadata of the images and save it:   1%|          | 30/4490 [00:00<01:32, 48.18it/s]

An error occurred while processing ../output/images/image_MAjtbFMV5_k.jpg: 'NoneType' object has no attribute 'items'


Get all metadata of the images and save it:   1%|▏         | 57/4490 [00:01<02:11, 33.79it/s]

An error occurred while processing ../output/images/image_revxuIor0nY.jpg: 'NoneType' object has no attribute 'items'


...

An error occurred while processing ../output/images/image_L6fROjb21U0.jpg: 'NoneType' object has no attribute 'items'


Get all metadata of the images and save it: 100%|█████████▉| 4484/4490 [11:48<00:02,  2.64it/s]

An error occurred while processing ../output/images/image_A8iLzX6OddM.jpg: 'NoneType' object has no attribute 'items'


Get all metadata of the images and save it: 100%|██████████| 4490/4490 [11:50<00:00,  6.32it/s]

Checkpoint removed successfully


# How to look at the metadata (pickle format)
This is the way to look at the metadata information of an image in pickle format.

In [ ]:
# Get the first file of metadata directory
metadata_file = os.listdir(metadata_path)[0]
# Load the metadata
with open(os.path.join(metadata_path, metadata_file), 'rb') as f:
    metadata = pickle.load(f)
# Print the metadata
print(metadata)

# How to look at the metadata (json format)
This is the way to look at the metadata information of an image in json format.

In [ ]:
# Get the first file of metadata directory
metadata_file = os.listdir(metadata_path)[0]
# Load the metadata
with open(os.path.join(metadata_path, metadata_file), 'r') as f:
    metadata = json.load(f)
# Print the metadata
print(metadata)

# How to look at the metadata (sqlite format)
This is the way to look at the metadata information of an image in sqlite database format.

In [34]:
# Open a connection to the database
conn = sqlite3.connect(os.path.join(metadata_path, 'metadata.db'))
# Create a cursor
c = conn.cursor()
# Get a name of the first file in the images directory
metadata_file = os.listdir(images_path)[0]
# Get the metadata of the first file
c.execute("SELECT * FROM metadata WHERE filename=?", (metadata_file,))
# Print the metadata
#Convert result to format: filename: [{key: value}...]
metadata = c.fetchall()
result = {}
for row in metadata:
    if row[0] not in result:
        result[row[0]] = []
    result[row[0]].append({row[1]: row[2]})

print(result)

# Close the connection
conn.close()

{'image_TCfCXuhgG4c.jpg': [{'ResolutionUnit': '2'}, {'ExifOffset': '214'}, {'Make': 'Canon'}, {'Model': 'Canon EOS Rebel T6'}, {'Software': 'Adobe Photoshop Camera Raw 10.5 (Windows)'}, {'DateTime': '2018:12:24 14:33:30'}, {'XResolution': '72.0'}, {'YResolution': '72.0'}, {'ExifVersion': "b'0230'"}, {'ShutterSpeedValue': '5.643856'}, {'ApertureValue': '4.33985'}, {'DateTimeOriginal': '2018:12:22 02:45:53'}, {'DateTimeDigitized': '2018:12:22 02:45:53'}, {'ExposureBiasValue': '0.0'}, {'MaxApertureValue': '4.25'}, {'MeteringMode': '5'}, {'ColorSpace': '65535'}, {'Flash': '16'}, {'FocalLength': '33.0'}, {'ExposureMode': '0'}, {'WhiteBalance': '0'}, {'SceneCaptureType': '0'}, {'FocalPlaneXResolution': '5728.176795580111'}, {'FocalPlaneYResolution': '5808.403361344538'}, {'FocalPlaneResolutionUnit': '2'}, {'SubsecTimeOriginal': '00'}, {'SubsecTimeDigitized': '00'}, {'ExposureTime': '0.02'}, {'FNumber': '4.5'}, {'ExposureProgram': '0'}, {'CustomRendered': '0'}, {'ISOSpeedRatings': '2000'}, {'